In [79]:
from github import Github
import os
import time
import pandas as pd

In [ ]:
department = 'vnpay'
private_token = ''
host = "https://github.com" 

In [81]:
g = Github(login_or_token=private_token)

# Folder to store export files
info_folder = os.path.join('info', department)
os.makedirs(info_folder, exist_ok=True)


In [82]:
# Lấy các organizations của user
organizations = g.get_user().get_orgs()
list_organizations = []

In [83]:
# Thu thập thông tin các organizations
for org in organizations:
    list_organizations.append({
        'id': org.id,
        'name': org.login,
        'path': org.login,
        'url': org.html_url,
    })

df_organizations = pd.DataFrame(list_organizations)
df_organizations

,id,name,path,url
0,129816109,blc-onchain,blc-onchain,https://github.com/blc-onchain
1,188412566,ntkongdm,ntkongdm,https://github.com/ntkongdm


In [84]:
# Lưu thông tin các organization vào CSV
file_org = os.path.join(info_folder, 'organization-export.csv')
df_organizations.to_csv(file_org)

In [85]:
# Lấy repository cá nhân của user
user = g.get_user()
personal_repositories = user.get_repos()
list_repositories = []

In [86]:
# List all org
for org in organizations:
    repositories = org.get_repos()
    for repo in repositories:
        # Tách phần tên tổ chức (org) từ full_name
        org_name, repo_name = repo.full_name.rsplit('/', 1)

        # Tạo thư mục cho tổ chức (nếu chưa tồn tại)
        repo_folder = os.path.join(info_folder, org_name)
        os.makedirs(repo_folder, exist_ok=True)

        # Đường dẫn tệp tar.gz repository
        repo_file = os.path.join(repo_folder, f"{repo_name}")

        # URL để tải repository ở dạng tar.gz (thay cho .zip)
        repo_url = f"https://github.com/{repo.full_name}"
        list_repositories.append({
            'id': repo.id,
            'name': repo.name,
            'name_with_namespace': repo.full_name,
            'folder': repo_folder,
            'project_file': repo_file,
            'group': repo.organization.login,
            'url': repo_url
        })
        # Dùng curl để tải file về
        os.system(f"git clone https://{private_token}@github.com/{repo.full_name}.git {repo_file}")
        # os.system(f"curl -L {repo_url} -o {repo_file}")
        print(f"Downloaded {repo_name} to {repo_file}: {repo_url}")


Cloning into 'info/vnpay/blc-onchain/bot-chat-gpt'...


Downloaded bot-chat-gpt to info/vnpay/blc-onchain/bot-chat-gpt: https://github.com/blc-onchain/bot-chat-gpt


Cloning into 'info/vnpay/ntkongdm/temp'...


Downloaded temp to info/vnpay/ntkongdm/temp: https://github.com/ntkongdm/temp


In [87]:
# # Thu thập thông tin các repository cá nhân
# for repo in personal_repositories:

#     # Tách phần tên người dùng (user) từ full_name
#     user_name, repo_name = repo.full_name.rsplit('/', 1)

#     # Tạo thư mục cho người dùng (nếu chưa tồn tại)
#     repo_folder = os.path.join(info_folder, user_name)
#     os.makedirs(repo_folder, exist_ok=True)

#     # Đường dẫn tệp zip repository
#     repo_file = os.path.join(repo_folder, f"{repo_name}.zip")

#     # URL để tải repository
#     repo_url = f"https://github.com/{repo.full_name}/archive/refs/heads/main.zip"  # Giả sử nhánh 'main' là mặc định

#     # Dùng curl để tải file về
#     os.system(f"curl -L {repo_url} -o {repo_file}")
#     list_repositories.append({
#         'id': repo.id,
#         'name': repo.name,
#         'name_with_namespace': repo.full_name,
#         'folder': repo_folder,
#         'project_file': repo_file,
#         'group': repo.organization
#     })


#     print(f"Downloaded {repo.organization}/{repo_name} to {repo_file}")

# # Chuyển thông tin repository sang DataFrame

In [88]:
df_repositories = pd.DataFrame(list_repositories)
df_repositories

,id,name,name_with_namespace,folder,project_file,group,url
0,601876893,bot-chat-gpt,blc-onchain/bot-chat-gpt,info/vnpay/blc-onchain,info/vnpay/blc-onchain/bot-chat-gpt,blc-onchain,https://github.com/blc-onchain/bot-chat-gpt
1,888238368,temp,ntkongdm/temp,info/vnpay/ntkongdm,info/vnpay/ntkongdm/temp,ntkongdm,https://github.com/ntkongdm/temp


In [89]:
# Lưu thông tin repository vào CSV
file_repo = os.path.join(info_folder, 'repository-export.csv')
df_repositories.to_csv(file_repo)

print("Done exporting GitHub data")

Done exporting GitHub data
